# Hakai CTD Profile QA/QC Development Tool
This jupyter notebook was developed to provide a flexible tool for testing and improving Hakai's QA/QC workflow for CTD profile data. 

The tool loads Hakai's CTD dataset and allows the user to modify the default tests applied to the data so that different QA/QC thresholds can be evaluated. In addition, new tests can be developed and applied to the data to test thier effectiveness. 

## Load packages
The initial load of these packages may take considerable time. 

Some of the packages needed are already available on the google Colab Server (pandas, seaborn, matplotlib, json). Others need to be downloaded and installed from their repository on the Hakai and IOOS Github account with the Pypi(pip) package: 
- [hakai-api-client-python](https://github.com/HakaiInstitute/hakai-api-client-python)
- [ioos_qc](https://github.com/HakaiInstitute/ioos_qc@colab-compatible)(Hakai Institute Fork and colab-compatible branch) ioos_qc is actively in developpement. To minimize issues with new changes to the ioos branch we are using a forked version on the Hakai Domain.
- [hakai_qc](https://github.com/HakaiInstitute/hakai-profile-qaqc) This package is the main package developped to run the different tests on the Hakai CTD profile dataset. 
- ipywidgets which makes possible the interactive plotting tool!

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Get External packages
try:
    from hakai_api import Client
    from ioos_qc.config import QcConfig
    import hakai_qc
    import ipywidgets as widgets
    from ipywidgets import interact
    
except:
    # Install Hakai API Python Client
    !pip install git+https://github.com/HakaiInstitute/hakai-api-client-python.git
    from hakai_api import Client

    # Install ioos_qc
    !pip install git+https://github.com/HakaiInstitute/ioos_qc@colab-compatible
    from ioos_qc.config import QcConfig
    
    # Load local modules
    !pip install git+https://github.com/HakaiInstitute/hakai-profile-qaqc.git
    import hakai_qc
    
    !pip install ipywidgets
    import ipywidgets as widgets
    from ipywidgets import interact,interact_manual

!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


## Import Stations and CTD data
Below, Hakai oceanography stations are uploaded. The Hakai Station Master List is based on a CSV output of the [Hakai Oceanography Master Stations Map and Data](https://hakai.maps.arcgis.com/apps/webappviewer/index.html?id=38e1b1da8d16466bbe5d7c7a713d2678). In order to have the tests applied, new/missing stations will need to be added to the master list.

In [2]:
# Load Hakai Station List
hakai_stations = hakai_qc.get.hakai_stations()

The following code allows the user to select a station of interest (i.e. QU39) and then, using the API, downloads the processed CTD data for this station from the Hakai database. Note that the API query will need to be authorized, which requires following the link that appears below the code block (appears after the code block has been run) and copying/pasting the authentication URL into the provided box. The paste dropdown button does not work here, so use CNTRL-V to paste the URL into the box.

In [4]:
# Get Hakai CTD Data Download through the API
station = 'QU39'

variable_lists = hakai_qc.get.hakai_api_selected_variables()

# Let's just get the data from QU39
filterUrl = 'station='+station+'&status!=MISCAST&limit=-1'+'&fields='+','.join(variable_lists)
#filterUrl = 'station=QU39&status!=MISCAST&limit=-1'+fields
df, url = hakai_qc.get.hakai_ctd_data(filterUrl)
print(str(len(df))+' records found')

# Regroup profiles and sort them by pressure
group_variables = ['device_model','device_sn','ctd_file_pk','ctd_cast_pk','direction_flag']
df = df.sort_values(by=group_variables+['pressure'])

# Get Derived Variables
df = hakai_qc.utils.derived_ocean_variables(df)

# Just show the first few lines to have a look
df.head() # Show the top of the data frame

Please go here and authorize:
https://hecate.hakai.org/api/auth/oauth2?response_type=code&client_id=289782143400-1f4r7l823cqg8fthd31ch4ug0thpejme.apps.googleusercontent.com&state=wEDduPMTCDASN7mAOBpLfhcyK1zB1Z

Paste the full redirect URL here:
https://hecate.hakai.org/api/auth/oauth2/callback?state=wEDduPMTCDASN7mAOBpLfhcyK1zB1Z&code=4/0AY0e-g4A6I30x6aFgZ4JL3VQffQgz_uRvVdFX6IhPlETE5D3AnPxx59M4qKxDYKNEKqbfQ&scope=email%20profile%20openid%20https://www.googleapis.com/auth/userinfo.profile%20https://www.googleapis.com/auth/userinfo.email&authuser=0&hd=hakai.org&prompt=none
135861 records found


C:\Users\jessy\Anaconda3\lib\site-packages\gsw\_wrapped_ufuncs.py:696: RuntimeWarning:

invalid value encountered in ct_from_t



,ctd_file_pk,ctd_cast_pk,hakai_id,ctd_data_pk,filename,device_model,device_sn,work_area,cruise,station,...,sos_un,sos_un_flag,backscatter_beta,backscatter_beta_flag,cdom_ppb,cdom_ppb_flag,absolute salinity,conservative temperature,density,sigma0
117244,2745,7913,080217_2017-01-05T17:32:36.333Z,9169911,080217_20170105_1317,RBRconcerto,080217,QUADRA,QOMB,QU39,...,None,None,None,None,None,None,28.019280,6.838949,1021.857610,21.852974
117245,2745,7913,080217_2017-01-05T17:32:36.333Z,9169912,080217_20170105_1317,RBRconcerto,080217,QUADRA,QOMB,QU39,...,None,None,None,None,None,None,28.009184,6.862007,1021.851638,21.842369
117246,2745,7913,080217_2017-01-05T17:32:36.333Z,9169913,080217_20170105_1317,RBRconcerto,080217,QUADRA,QOMB,QU39,...,None,None,None,None,None,None,28.008935,6.854664,1021.856944,21.843039
117247,2745,7913,080217_2017-01-05T17:32:36.333Z,9169914,080217_20170105_1317,RBRconcerto,080217,QUADRA,QOMB,QU39,...,None,None,None,None,None,None,28.009692,6.854777,1021.862157,21.843617
117248,2745,7913,080217_2017-01-05T17:32:36.333Z,9169915,080217_20170105_1317,RBRconcerto,080217,QUADRA,QOMB,QU39,...,None,None,None,None,None,None,28.013262,6.863834,1021.868514,21.845340


## Hakai CTD Profile QA/QC Test Configuration

This step imports and configures (i.e sets test thresholds) the QA/QC tests that will be applied to the Hakai CTD data uploaded from the database. 

New tests can be added by following the structure within the code block below. For additional information on the different tests available, please look at the [ioos_qc webpage](https://ioos.github.io/ioos_qc/api/ioos_qc.html).

Test thresholds can be modified using the structure provided in the fluoroescence example below. Variable abbreviations, required to change the thresholds, will appear in a table once this code block has been run.


In [5]:
# Load default test parameters used right now!
qc_config = hakai_qc.get.json_config('hakai_ctd_profile.json')

#Test parameters can be modified below following the provided format for fluorescence 
#ex: This code sets the range outside of which fluorescence is considered suspect and implausible (fail) 
qc_config['flc']= {'qartod': {
                        'gross_range_test': {   
                            "suspect_span": [0, 70],
                            "fail_span": [-.5, 100],
                        }
                   }}

target = {'target_range':[1000]}
qc_config['position']['qartod']['location_test'].update(target)

# Displays the QC/QA test parameters in a table
hakai_qc.get.config_as_dataframe(qc_config)

Value
Variable              Module Test                   Input                                            
position              qartod location_test          bbox                         [-180, -90, 180, 90]
                                                    target_range                               [1000]
pressure              qartod gross_range_test       fail_span                              [0, 12000]
                                                    maximum_fail_depth_ratio                      1.1
                                                    maximum_suspect_depth_ratio                  1.05
                                                    suspect_span                           [0, 12000]
depth                 qartod gross_range_test       fail_span                              [0, 12000]
                                                    maximum_fail_depth_ratio                      1.1
                                                    maximum_suspect_depth_ratio                  1.05
                                                    suspect_span                           [0, 12000]
dissolved_oxygen_ml_l qartod gross_range_test       fail_span                                 [0, 20]
                                                    suspect_span                              [1, 15]
                             rate_of_change_test    threshold                                       3
                             spike_test             suspect_threshold                             0.5
                                                    fail_threshold                                  1
                             attenuated_signal_test suspect_threshold                             0.1
                                                    fail_threshold                               0.01
                                                    check_type                                  range
rinko_do_ml_l         qartod gross_range_test       fail_span                                 [0, 20]
                                                    suspect_span                              [1, 15]
                             rate_of_change_test    threshold                                       3
                             spike_test             suspect_threshold                             0.5
                                                    fail_threshold                                  1
                             attenuated_signal_test suspect_threshold                             0.1
                                                    fail_threshold                               0.01
                                                    check_type                                  range
turbidity             qartod gross_range_test       fail_span                           [-0.1, 10000]
                                                    suspect_span                            [0, 1000]
                             attenuated_signal_test suspect_threshold                            0.01
                                                    fail_threshold                              0.001
                                                    check_type                                  range
c_star_at             qartod attenuated_signal_test suspect_threshold                           0.002
                                                    fail_threshold                             0.0001
                                                    check_type                                  range
                             spike_test             suspect_threshold                             0.5
                                                    fail_threshold                                  1
par                   qartod gross_range_test       fail_span                          [-0.2, 100000]
                                                    suspect_span                           [0, 50000]
                

## Apply QA/QC Tests to CTD Profiles

The QA/QC tests and thresholds defined in the above code block/table are applied to the CTD data on a profile by profile basis. Only profiles from the pre-selected station will be evaluated. 

In [6]:
# Run all of the tests on each available profile
df = hakai_qc.run.tests_on_profiles(df,hakai_stations,qc_config)

QAQC QU39
  position
    qartod
      ('location_test', {'bbox': [-180, -90, 180, 90], 'target_range': [1000], 'target_lat': [50.0307000000001], 'target_lon': [-125.0992]})
  pressure
    qartod
      ('gross_range_test', {'suspect_span': [0, 280.83826311944904], 'fail_span': [0, 294.2115137441847], 'maximum_suspect_depth_ratio': 1.05, 'maximum_fail_depth_ratio': 1.1})
  depth
    qartod
      ('gross_range_test', {'suspect_span': [0, 278.25], 'fail_span': [0, 291.5], 'maximum_suspect_depth_ratio': 1.05, 'maximum_fail_depth_ratio': 1.1})
  dissolved_oxygen_ml_l
    qartod
      ('gross_range_test', {'fail_span': [0, 20], 'suspect_span': [1, 15]})
      ('rate_of_change_test', {'threshold': 3})
      ('spike_test', {'suspect_threshold': 0.5, 'fail_threshold': 1})
      ('attenuated_signal_test', {'suspect_threshold': 0.1, 'fail_threshold': 0.01, 'check_type': 'range'})
  rinko_do_ml_l
    qartod
      ('gross_range_test', {'fail_span': [0, 20], 'suspect_span': [1, 15]})
      ('rate_of_

## Review Results
###  Profile location versus target location
This section present the CTD profile qartod location_test result which is comparing the CTD drop location versus the target station coordinates as defined in the Hakai Station Master list. We present here all the drops that failed the test:

In [7]:
# Output all drops with flagged positions that exceed the distance threshold or do not display coordinates (NaN)
#  ignore rows where a depth value does not exist.
df[df['position_qartod_location_test']>1].dropna(
    axis=0,subset=['depth']).groupby(
    'hakai_id').first()[['position_qartod_location_test','station','latitude','longitude','measurement_dt']]

,position_qartod_location_test,station,latitude,longitude,measurement_dt
hakai_id,,,,,
01907674_2017-07-17T16:02:34Z,9.0,QU39,NaN,NaN,2017-07-17T16:05:50.163Z
01907674_2017-07-24T15:05:54Z,3.0,QU39,50.038057,-125.109211,2017-07-24T15:08:46.371Z
01907674_2017-08-08T15:09:00Z,9.0,QU39,NaN,NaN,2017-08-08T15:12:13.829Z
01907674_2017-08-08T16:34:48Z,9.0,QU39,NaN,NaN,2017-08-08T16:37:38.271Z
01907674_2018-01-09T18:14:24Z,9.0,QU39,NaN,NaN,2018-01-09T18:16:30.149Z
01907674_2018-12-04T17:11:03Z,9.0,QU39,NaN,NaN,2018-12-04T17:14:32.752Z
01907674_2018-12-05T19:56:38Z,9.0,QU39,NaN,NaN,2018-12-05T20:01:38.857Z
01907674_2018-12-19T17:30:29Z,9.0,QU39,NaN,NaN,2018-12-19T17:32:10.502Z
01907674_2019-07-03T17:47:15Z,3.0,QU39,50.190470,-124.996663,2019-07-03T17:50:41.419Z


In [8]:
# Display the CTD profiles with flagged GPS coordinates
m = hakai_qc.get.flag_result_map(df.dropna(axis=0,subset=['latitude','longitude','depth']),
                                 flag_variable='position_qartod_location_test')
m

### Review Profile test flags
We present here an interactive way of plotting and reviewing the different flags. Just run the following cell and interact with the figure below.

#### First let's review statistically the results of the tests!

In [236]:
# Get the flag columns and hakai_id, but ignore the direction flag column
test_list = sorted(df.filter(regex='_flag$|_test$').columns.tolist())
test_list.append('hakai_id')
test_list.remove('direction_flag')

# Get Flag columns, stack them all and add a count column
df_test = df[test_list].set_index('hakai_id').stack().reset_index().rename({'level_1':'Test',0:'Flag'},axis=1)
df_test['Count'] = 1

#Regroup data by test and flag and count how many values there is per flag
df_result = df_test.groupby(['Test','Flag'])['Count'].count().dropna().reset_index()

# Make sure that all the flags are in int and convert to str format for plotly, sort by flag and test
df_result['Flag'] =df_result['Flag'].astype(int).astype(str)
df_result = df_result.sort_values(['Flag','Test'])

# Create bar plot
fig = px.bar(df_result,x='Count',y='Test',orientation='h',color='Flag',
       color_discrete_sequence=['green','yellow','orange','red','purple'])
fig.update_layout(
    autosize=False,
    width=950,
    height=1000)

#### Review one profile at the time

In [100]:
variable_list = set(qc_config.keys())-{'position'}
test_list = sorted(df.filter(regex='_flag$|_test$').columns.tolist())
test_list.append("None")
qartod_color = {1:'green',2:'yellow',3:'orange',4:'red',9:'purple','1':'green','2':'yellow','3':'orange','4':'red','9':'purple'}
dir = {'d':'downcast','u':'upcast'}
    
print('Select Flagged Variable(s) to consider in the following list:')
@interact
def which_profiles_to_look_at(
    FlagType=widgets.Dropdown(options=test_list,value='None',
                              description='Test to review',disabled=False),
    ConsideredFlag = widgets.SelectMultiple(options=[1,2,3,4,9],value=[1,2,3,4,9],
                                            description='Flag To Consider',disabled=False)):
      
    if FlagType=='None':
        hakai_id_list = df['hakai_id'].unique().tolist()
    else:
        hakai_id_list = df[df[FlagType].isin(ConsideredFlag)]['hakai_id'].unique()
    
    print(str(len(hakai_id_list))+' profiles are available')
    
    @interact
    def plot_profile(hakai_id=hakai_id_list,
                     Ocean_Variables=widgets.SelectMultiple(options=variable_list-{'depth','pressure'},
                                                            value=['temperature','salinity','dissolved_oxygen_ml_l'],
                                                            description='Ocean Variable',
                                                            disabled=False), 
                 
                     y_axis=widgets.Dropdown(options=variable_list,value='depth',description='Y Axis Variable',
                                         disabled=False),
                     downcast=widgets.Checkbox(value=True,description='Downcast',
                                             disabled=False),
                     upcast=widgets.Checkbox(value=True,description='Upcast',
                                             disabled=False),
                     par_log_scale=widgets.Checkbox(value=True,description='PAR Log Scale',
                                             disabled=False)):

        cast_direction=[]
        if downcast:
            cast_direction.append('d')
        if upcast:
            cast_direction.append('u')

        df_temp = df[df['hakai_id']==hakai_id].sort_values(['direction_flag','depth'])
        vars = Ocean_Variables

        #Create Subplots
        fig = make_subplots(rows=1,cols=len(vars), shared_yaxes=True,
                            horizontal_spacing=0.01)
        kk=1
        for var in vars:
            for direction_flag in cast_direction:
                for flag, color in qartod_color.items():
                    df_flag = df_temp[(df_temp[var+'_qartod_flag']==flag) & (df_temp['direction_flag']==direction_flag)]

                    if len(df_flag):
                        if direction_flag is 'u':
                            marker_dict = dict(color=color,symbol='x')
                        else:
                            marker_dict = dict(color=color)

                        fig.add_trace(
                        go.Scatter(x=df_flag[var],
                                y=df_flag[y_axis],
                                mode='markers',
                                marker=marker_dict,# df_temp[var+'_qartod_flag'],
                                text=df_flag[var+'_flag_description'],
                                name=var+' '+dir[direction_flag]+' FLAG:'+str(flag)),
                            row=1,col=kk)

            if var in ['par'] and par_log_scale: # Make PAR x axis log
                fig.update_xaxes(type="log",row=1,col=kk)    
            fig.update_xaxes(title=var, row=1, col=kk)
            kk=kk+1

        # Add stuff around each figures
        fig.update_yaxes(title_text=y_axis,row=1,col=1)
        fig.update_yaxes(autorange="reversed",linecolor='black',mirror=True,ticks='outside',showline=True)
        fig.update_xaxes(mirror=True,ticks='outside',showline=True,tickangle=45,linecolor='black')
        fig.update_layout(height=800, width=2000,showlegend=True, 
                          title_text='Hakai ID: '+hakai_id+' Site: '+df_temp['station'].unique()[0])
        return fig.show()
    
    return 


Select Flagged Variable(s) to consider in the following list:


interactive(children=(Dropdown(description='Test to review', index=61, options=('backscatter_beta_flag', 'bott…

#### Plot data as scatter time series

In [142]:

maxPressure = max(df['pressure'])
middlePressure = round(maxPressure/2)
deltaPresure = round(.15*maxPressure)

@ interact
def plot_scatter(var=widgets.Dropdown(options=variable_list-{'depth','pressure'},
                                            value='dissolved_oxygen_ml_l',
                                            description='Ocean Variable',
                                            disabled=False),
                pressure=widgets.IntRangeSlider(value=[middlePressure-deltaPresure,middlePressure+deltaPresure],
                                                min=0,max=max(df['pressure']),step=1,description='Pressure Range:',
                                                disabled=False,continuous_update=False,orientation='horizontal',
                                                readout=True,readout_format='d')):


    # Plot scatter
    df_plot = df[(df['direction_flag']=='d')& (df[var+'_qartod_flag']==1) & (df[var].notna())]
    fig = px.scatter(df_plot,x='measurement_dt',y='depth',color=var)
    fig.update_yaxes(autorange="reversed")

    df_plot_depth = df_plot[(df_plot['pressure']>pressure[0]) & (df_plot['pressure']<pressure[1])]
    fig2 = px.scatter(df_plot_depth,x='measurement_dt',y=var,color='pressure')

    fig.show()
    fig2.show()
    return

interactive(children=(Dropdown(description='Ocean Variable', index=8, options=('salinity', 'sigma0', 'temperat…